# Chapter 8: Robustness and Performance

## Item 65: Take Advantage of Each Block in `try/except/else/finally`

## Item 66: Consider `contextlib` and `with` Statements for Reusable `try/finally` Behavior

The `with` statement in Python is used to indicate when code is running in a special context.  For example, mutual-exclusion locks can be used in `with` statements to indicate that the indented code block runs only while the lock is held:

In [3]:
from threading import Lock

lock = Lock()
with lock:
    # do something
    pass

The above example is the same as using a try/finally construction except it will automatically `release` the lock and ensure you don't have to worry about doing it manually.

The context manager passed to a `with` statement may also return an object.  This object is assigned to a local variable in the `as` part of a compound statement.

To enable your own funcitons to supply values for `as` targets, all you need to do is `yield` a value from your context manager:

## Item 67: Use `datetime` Instead of `time` for Local Clocks

tl;dr avoid using the `time` module for translating between different time zones.  

Use the `datetime` built-in module along with the `pytz` community module to reliably convert between times in different time zones.  

Always represent time in UTC and do conversions to local time as the very fina lstep before presentation.

## Item 68:  Make `pickle` Reliable with `copyreg`

The `pickle` built-in module can serialize Python objects into a stream of bytes and deserialize bytes back into objects.  Pickled byte streams shouldn't be used to communicate between untrusted parties.  The purpose of `pickle` is to let you pass Python objects between programs that you control over binary channels.  

## Item 69:  Use `decimal` When Precision Is Paramount



In [6]:
from decimal import Decimal

rate = 1.45
seconds = 3*60 + 42
cost = rate * seconds / 60
print(cost)

rate = Decimal('1.45')
seconds = Decimal(3*60 + 42)
cost = rate * seconds / Decimal(60)
print(cost)

5.364999999999999
5.365


## Item 70:  Profile Before Optimizing

Python provides a built-in *profiler* for determining which parts of a program are responsible for execution time.  This means that you can focus your optimization efforts on the biggest sources of trouble and ignore parts of the program that don't impact speed.

In [8]:
import logging
from pprint import pprint
from sys import stdout as STDOUT

def insertion_sort(data):
    result = []
    for value in data:
        insert_value(result, value)
    return result


# Example 2
def insert_value(array, value):
    for i, existing in enumerate(array):
        if existing > value:
            array.insert(i, value)
            return
    array.append(value)


# Example 3
from random import randint

max_size = 10**4
data = [randint(0, max_size) for _ in range(max_size)]
test = lambda: insertion_sort(data)


# Example 4
from cProfile import Profile

profiler = Profile()
profiler.runcall(test)


# Example 5
from pstats import Stats

stats = Stats(profiler)
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()


         20003 function calls in 0.818 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.818    0.818 <ipython-input-8-319857f273fc>:26(<lambda>)
        1    0.001    0.001    0.818    0.818 <ipython-input-8-319857f273fc>:5(insertion_sort)
    10000    0.809    0.000    0.817    0.000 <ipython-input-8-319857f273fc>:13(insert_value)
     9987    0.008    0.000    0.008    0.000 {method 'insert' of 'list' objects}
       13    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Looking at the profiler statistics table above, I can see that the biggest use of CPU in my test is the cumulative time spent in the `insert_value` function.

## Item 71: Prefer `deque` for Producer-Consumer Queues

Many people use lists for queues, but the issue is that in order to remove an item from the left of the list requires shifting all other elements of the list.  Python provides the `deque` class from the `collections` built-in module to solve this problem.  `deque` is a *double-ended queue* implementation.

## Item 72: Consider Searching Sorted Sequences with `bisect`

Searching sorted data contained in a `list` takes linear time using the `index` method or a `for` loop with simple comparisons.  

The `bisect` built-in module's `bisect_left` function takes logarithmic time to search for values in sorted lists, which can be orders of magnitude faster than other approaches.  

## Item 73:  Know how to use `heapq` for Priority Queues

Often times, instead of needing a FIFO queue, you'll need a program to process items in order of relative importance instead.  To accomplish this, a **priority queue** is the right tool for the job.

A **heap** is a data structure that allows for a `list` of items to be maintained where the computational complexity of adding a new item or removing the smallest item has logarithmic computational complexity.  

The `heapq` module requires items in the priority queue to be comparable and ahve a natural sort order, which requires special methods like `__lt__` to be defined for classes.  


## Item 74:  Consider `memoryview` and `bytearray` for Zero-Copy Interactions with `bytes`  

Python's built-in `memoryview` type exposes CPython's high-performance **buffer protocol** to programs.  The buffer protocol is a low-level C API that allows the Python runtime and C extensions to access the underlying data buffers that are behind objects like `bytes` instances.  The best part about `memoryview` instances is that slicing them results in another `memoryview` instance without copying the underlying data.

By enabling `zero-copy` operations, `memoryview` can provide enormous speedups for code that needs to quickly process large amounts of memory, such as numerical C extensions like `NumPy` and other I/O bound programs.  

In [9]:
data = b'shave and a haricut, two bits'
view = memoryview(data)
chunk = view[12:19]
print(chunk)
print(f'Size: {chunk.nbytes}')
print(f'Data in view: {chunk.tobytes()}')
print(f'Underlying data: {chunk.obj}')


Size: 7
Data in view: b'haricut'
Underlying data: b'shave and a haricut, two bits'
